In [ ]:
import numpy as np
import pandas as pd
import nltk
nltk.download('punkt') # one time execution
import re
import wikipedia
import wikipediaapi
import sys
from nltk.tokenize import sent_tokenize

from tkinter import *


[nltk_data] Downloading package punkt to C:\Users\Shubham
[nltk_data]     Gaurav\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:

dat = input("enter the phrase")
N =input("enter total no of relevant sentences required")
N = int(N)

try:
    p =wikipedia.page(dat)
    
    content = p.content # Content of page.
    #content

    sentences = []
    sentences.append(sent_tokenize(content))

    sentences = [y for x in sentences for y in x] # flatten list
    #sentences[:5]   

    clean_sentences = pd.Series(sentences).str.replace("[^a-zA-Z]", " ") #remove integers and special symbols
    clean_sentences = [s.lower() for s in clean_sentences] #make sentence in lowercase
    #clean_sentences[:5]


    nltk.download('stopwords')
    from nltk.corpus import stopwords
    stop_words = stopwords.words('english')
    def remove_stopwords(sen):
        sen_new = " ".join([i for i in sen if i not in stop_words])
        return sen_new

    clean_sentences = [remove_stopwords(r.split()) for r in clean_sentences]
    #clean_sentences[:5]

    word_embeddings = {}
    f = open('C:/Users/Shubham Gaurav/Desktop/project/glove.6B.100d.txt', encoding='utf-8')
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        word_embeddings[word] = coefs
    f.close()


    sentence_vectors = []
    for i in clean_sentences:
        if len(i) != 0:
            v = sum([word_embeddings.get(w, np.zeros((100,))) for w in i.split()])/(len(i.split())+0.001)
        else:
            v = np.zeros((100,))
        sentence_vectors.append(v)

    type(sentence_vectors[1])

    sim_mat = np.zeros([len(sentences), len(sentences)])

    from sklearn.metrics.pairwise import cosine_similarity
    for i in range(len(sentences)):
        for j in range(len(sentences)):
            if i!= j:
                sim_mat[i][j] = cosine_similarity(sentence_vectors[i].reshape(1,100), sentence_vectors[j].reshape(1,100))


    import networkx as nx

    nx_graph = nx.from_numpy_matrix(sim_mat)
    scores = nx.pagerank(nx_graph) 

    ranked_sentences = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)
    
    max_size = len(ranked_sentences)

    i = 0
    k = 0
    while i<N:
    
        if k==max_size:
            print("The maximum number of relevant sentences available is", i)
            break
        L=re.findall(r'\b\d+\b', ranked_sentences[k][1])
        #print(ranked_sentences[i][1])
        L=list(set(L))
        if L:
            j=0
            p=0
            while j<len(L):
                if len(L[j])==4:
                    print(L[j])
                    j +=1
                    p=1

                else:
                    j +=1
            if p==1:
                i +=1
                print(ranked_sentences[k][1])
                print("\n")
        k+=1
        
except:
    print("No article found")
    

enter the phrasemessi
enter total no of relevant sentences required5
[nltk_data] Downloading package stopwords to C:\Users\Shubham
[nltk_data]     Gaurav\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
